In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# from datetime import datetime
import datetime
from dateutil.relativedelta import relativedelta


load_dotenv()


hostname=os.getenv("hostname")
database=os.getenv("database")
username=os.getenv("username")
pwd=os.getenv("pwd")
port_id=os.getenv("port_id")
host = os.getenv("host")

import warnings
warnings.filterwarnings('ignore')


In [2]:


conn_string = f"postgresql://{username}:{pwd}@{host}:{port_id}/{database}"
db =  create_engine(url=conn_string)

transactions = pd.read_sql("SELECT * FROM transactions", db)

transactions.head()

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId,alpha,senator_id
0,2020-04-14,Spouse,DOW,Dow Inc.,Stock,Sale (Full),"$15,001 - $50,000",--,Kelly Loeffler,https://efdsearch.senate.gov/search/view/ptr/2...,2020-05-01,2198,137,None,137
1,2020-04-07,Joint,HELE,Helen of Troy Limited,Stock,Sale (Full),"$15,001 - $50,000",--,Kelly Loeffler,https://efdsearch.senate.gov/search/view/ptr/2...,2020-05-01,2317,137,None,137
2,2018-12-06,Self,PFE,Pfizer Inc.,Stock,Sale (Full),"$1,001 - $15,000",--,"Jerry Moran,",https://efdsearch.senate.gov/search/view/ptr/d...,2019-05-15,3451,119,None,119
3,2017-09-26,Spouse,T,AT&T Inc.,Stock,Sale (Full),"$1,001 - $15,000",sep,Pat Roberts,https://efdsearch.senate.gov/search/view/ptr/8...,2017-09-29,5683,133,None,133
4,2014-06-19,Spouse,CR,Crane Co. (NYSE),Unknown,Sale (Full),"$1,001 - $15,000",--,Susan M Collins,https://efdsearch.senate.gov/search/view/ptr/9...,2014-06-27,9111,106,None,106


In [3]:
senators = pd.read_sql("SELECT * FROM senator_info", db)
senators.head()

,row,senator,senatorId,party
0,1,Thomas H Tuberville,100,Republican
1,2,Thomas R Carper,101,Democratic
2,3,Daniel S Sullivan,102,Republican
3,4,Rick Scott,103,Republican
4,5,John Boozman,104,Republican


In [32]:
t = transactions.copy()

In [33]:
t[t.type=='Purchase'].asset_type.value_counts()

Stock    3420
Name: asset_type, dtype: int64

In [34]:
p = t[t.type=='Purchase']
p.head()

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date,id,senatorId,alpha,senator_id
5,2020-03-30,Joint,MU,"Micron Technology, Inc.",Stock,Purchase,"$1,001 - $15,000",--,"David A Perdue , Jr",https://efdsearch.senate.gov/search/view/ptr/0...,04/05/2020,2490,136,0.0,136
6,2016-08-22,Self,HAS,Hasbro Inc.,Stock,Purchase,"$50,001 - $100,000",--,John Hoeven,https://efdsearch.senate.gov/search/view/ptr/4...,08/29/2016,6965,125,14.68,125
4975,2020-11-17,Joint,W,Wayfair Inc.,Stock,Purchase,"$15,001 - $50,000",R,Pat Roberts,https://efdsearch.senate.gov/search/view/ptr/b...,12/03/2020,1338,133,28.15,133
4976,2020-08-21,Spouse,SQ,"Square, Inc.",Stock,Purchase,"$15,001 - $50,000",--,Ron L Wyden,https://efdsearch.senate.gov/search/view/ptr/3...,08/29/2020,1565,163,62.47,163
4977,2020-08-24,Spouse,SQ,"Square, Inc.",Stock,Purchase,"$15,001 - $50,000",--,Ron L Wyden,https://efdsearch.senate.gov/search/view/ptr/2...,09/24/2020,1481,163,63.66,163


In [35]:
# p.senator.value_counts()
grp = pd.DataFrame(p[['senator', 'senatorId']].value_counts())
grp.reset_index(inplace=True)
grp.columns = ['senator','senatorId','purchases']
grp = grp[grp['purchases']>2]
grp

,senator,senatorId,purchases
0,"David A Perdue , Jr",136,1228
1,Thomas R Carper,101,298
2,Sheldon Whitehouse,123,289
3,Pat Roberts,133,264
4,Shelley M Capito,110,226
5,Thomas H Tuberville,100,191
6,Ron L Wyden,163,190
7,John Hoeven,125,108
8,"Jerry Moran,",119,95
9,James M Inhofe,132,85


In [36]:
# p.sort_values(by="transaction_date", ascending=False)

In [9]:
# p[p['senatorId']==1 and p['alpha'].notnull()].alpha.mean()

# Summary Stats Needed: 
<ul>
<li>Max Alpha</li>
<li>Highest Mean Alpha </li>
<li>Most Purchased Stock</li>
</ul>

In [44]:
#maxAlpha by Senator

# p.sort_values(by="alpha", ascending=False)
startLen = p.shape[0]

errorOutput = 99999

def stringToFloat(val):
    rez =0
    try:
        rez = float(val)
    except ValueError:
        rez = errorOutput
    return rez


In [46]:
# testVals  = p.alpha.to_list()[:45]

# testVals[0]


# stringToFloat('0.0')

In [47]:

p['alpha'] = p.alpha.apply(lambda row: stringToFloat(row))

p = p[p['alpha']!=errorOutput]

print(startLen, p.shape[0])

3420 2917


In [62]:
alphaSer =  p.groupby(['senatorId'])['alpha'].mean().sort_values(ascending=False)

maxAlphaSen = round(alphaSer.values[0], 2)
maxAlphaSenId = alphaSer.index[0]

maxSenName = p[p['senatorId']==maxAlphaSenId ].senator.values[0]

maxAlphaSenatorString = f"senator: {maxAlphaSen}, alpha: {maxAlphaSen}"
maxAlphaSenatorString

'senator: 18.56, alpha: 18.56'

In [63]:
resultsDict = {"maxAlphaSenator": maxAlphaSenatorString}

In [75]:
# highest alpha overall

maxAlphaRow = dict(p.loc[p.alpha.idxmax()])

highestAlphaOverallString = f"senator: {maxAlphaRow['senator']}, stock:{maxAlphaRow['ticker']}, date:{maxAlphaRow['transaction_date']}, alpha: {maxAlphaRow['alpha']}"
# highestAlphaOverallString 

resultsDict['highestAlphaOverall'] = highestAlphaOverallString 

resultsDict

{'maxAlphaSenator': 'senator: 18.56, alpha: 18.56',
 'highestAlphaOverall': 'senator: Thomas R Carper, stock:SI, date:2020-11-20, alpha: 450.03'}

In [77]:
# Most traded stock

purchaseSer = p.ticker.value_counts()
purchaseTicker = purchaseSer.index[0]
purchaseFreq= purchaseSer.values[0]

purchaseString = f"ticker:{purchaseTicker}, purchases: {purchaseFreq}"

resultsDict['purchaseFreq'] = purchaseString

resultsDict

{'maxAlphaSenator': 'senator: 18.56, alpha: 18.56',
 'highestAlphaOverall': 'senator: Thomas R Carper, stock:SI, date:2020-11-20, alpha: 450.03',
 'purchaseFreq': 'ticker:AAPL, purchases: 88'}

In [86]:
categories = resultsDict.keys()
results = resultsDict.values()

summaryStats = {'category':categories, 'result':results}
summaryStats = pd.DataFrame(summaryStats)
summaryStats

,category,result
0,maxAlphaSenator,"senator: 18.56, alpha: 18.56"
1,highestAlphaOverall,"senator: Thomas R Carper, stock:SI, date:2020-..."
2,purchaseFreq,"ticker:AAPL, purchases: 88"


# Post to db

In [87]:
import psycopg2 

conn=None
cur=None 
db = None

def postDataFrameDB():
    try:
        conn_string = f"postgresql://{username}:{pwd}@{host}:{port_id}/{database}"
        db =  create_engine(url=conn_string)
        conn=db.connect()

        summaryStats.to_sql('summary_stats', con=conn, if_exists='replace', index=False)

        connPsy = psycopg2.connect(conn_string)
        cur = connPsy.cursor()
        connPsy.autocommit = True

        sqlString = """
        select * from summary_stats; 
        """
        cur.execute(sqlString)

        records = cur.fetchall()
        for row in records:
            print(row)

    except Exception as err:
        print(err)
    finally: 
        if cur is not None: 
            cur.close()
        if conn is not None: 
            conn.close()

In [88]:
postDataFrameDB()

('maxAlphaSenator', 'senator: 18.56, alpha: 18.56')
('highestAlphaOverall', 'senator: Thomas R Carper, stock:SI, date:2020-11-20, alpha: 450.03')
('purchaseFreq', 'ticker:AAPL, purchases: 88')


In [ ]:
#explain this functino 